In [ ]:
%load_ext lab_black

In [ ]:
import os

os.chdir("..")

In [ ]:
import gzip
import swifter
import numpy as np
import pandas as pd
from scipy.sparse import csr_array

In [ ]:
dataset_names = ["enron", "kos", "nytimes", "pubmed"]
docword_paths = [
    os.path.join("resources", "data", f"docword.{name}.txt.gz")
    for name in dataset_names
]
vocab_paths = [
    os.path.join("resources", "data", f"vocab.{name}.txt") for name in dataset_names
]

In [ ]:
def convert_raw_data_to_bow_vectors(name, docword_path):
    df = pd.read_csv(docword_path, compression="gzip", skiprows=3, header=None, sep=" ")
    df.columns = ["doc_id", "word_id", "count"]
    max_id = df.word_id.max()

    def flatten_group(group):
        ids = group["word_id"].values
        counts = group["count"].values
        bow_vector = np.zeros(max_id)
        bow_vector[ids - 1] = counts
        bow_vector = csr_array(bow_vector)
        return bow_vector

    df = (
        df.groupby("doc_id")
        .apply(flatten_group)
        .reset_index(drop=False)
        .rename(columns={0: "bow_vector"})
    )

    df["target"] = name
    df.to_pickle(os.path.join("resources", "data", f"joined.{name}.pkl"))
    del [df]

In [ ]:
convert_raw_data_to_bow_vectors(dataset_names[0], docword_paths[0])

In [ ]:
pd.read_pickle(os.path.join("resources", "data", f"joined.{dataset_names[0]}.pkl"))

In [ ]:
pd.read_pickle(os.path.join("resources", "data", f"joined.{dataset_names[0]}.pkl")).loc[
    0, "bow_vector"
].toarray()